[Back to Index](index.ipynb)

## Generate Summary Statistics for Sacramento County

Begin by importing all python modules we will need. For this exercise we will use Pandas.

In [30]:
import requests
import pandas as pd
from datetime import datetime
import itertools
import calendar

### 1. Create a Pandas DataFrame

Now, let's create a new Pandas DataFrame from a timeseries. Let's get the tiemseries for Annual Average Maximum Temperature for one model (CNRM-CM5) and one scenario (RCP 4.5) for Alameda County.

A DataFrame is a 2D data structure, kind of like an Excel Spreadsheet tab.

In [74]:
# Create an empty dataframe to hold data from 4 models
index = pd.date_range(datetime(2006, 1, 1), periods=94, freq='AS')
projections_df = pd.DataFrame(index=index)

# Make a combined list of GCMs, scenarios, climate variables for looping
climvar = ['tasmax', 'tasmin', 'pr']
period = ['year']
models = ['CanESM2', 'CNRM-CM5', 'HadGEM2-ES', 'MIROC5']
scenarios = ['rcp45', 'rcp85']
zipped = itertools.product(climvar, period, models, scenarios)

# Request header
headers = {'ContentType': 'json'}

def get_projections():
    for item in zipped:
        
        slug = '_'.join(item)
        url = 'http://api.cal-adapt.org/api/series/' + slug + '/rasters/?pagesize=94&stat=mean&ref=/api/counties/34/'
        response = requests.get(url, headers=headers)

        # Get data
        if response.status_code == requests.codes.ok:
            print('Processing:', slug)
            data = response.json()
            # Create temp dataframe
            df = pd.DataFrame(data['results'])
            # Change format of `event` field to datetime
            df['event'] = pd.to_datetime(df['event'], format='%Y-%m-%d')
            # Set event field as index
            df = df.set_index(['event'])
            # The data from API should be sorted, but sort anyway
            df = df.sort_index()
            series = df.image
            column_name = slug.replace("-", "").lower()
            projections_df[column_name] = series.values
        else:
            print('Failed:', slug)

### 2. Explore the data in a DataFrame

Uncomment each one and examine the output

In [75]:
get_projections()

Processing: tasmax_year_CanESM2_rcp45
Processing: tasmax_year_CanESM2_rcp85
Processing: tasmax_year_CNRM-CM5_rcp45
Processing: tasmax_year_CNRM-CM5_rcp85
Processing: tasmax_year_HadGEM2-ES_rcp45
Processing: tasmax_year_HadGEM2-ES_rcp85
Processing: tasmax_year_MIROC5_rcp45
Processing: tasmax_year_MIROC5_rcp85
Processing: tasmin_year_CanESM2_rcp45
Processing: tasmin_year_CanESM2_rcp85
Processing: tasmin_year_CNRM-CM5_rcp45
Processing: tasmin_year_CNRM-CM5_rcp85
Processing: tasmin_year_HadGEM2-ES_rcp45
Processing: tasmin_year_HadGEM2-ES_rcp85
Processing: tasmin_year_MIROC5_rcp45
Processing: tasmin_year_MIROC5_rcp85
Processing: pr_year_CanESM2_rcp45
Processing: pr_year_CanESM2_rcp85
Processing: pr_year_CNRM-CM5_rcp45
Processing: pr_year_CNRM-CM5_rcp85
Processing: pr_year_HadGEM2-ES_rcp45
Processing: pr_year_HadGEM2-ES_rcp85
Processing: pr_year_MIROC5_rcp45
Processing: pr_year_MIROC5_rcp85


In [205]:
import calendar

# Create an empty dataframe to hold data from 4 models
index = pd.date_range(datetime(1950, 1, 1), periods=64, freq='AS')
observed_df = pd.DataFrame(index=index)

# Make a combined list of GCMs, scenarios, climate variables for looping
climvar = ['tasmax', 'tasmin', 'pr']
period = ['year']
zipped = itertools.product(climvar, period, ['livneh'])

# Request header
headers = {'ContentType': 'json'}

def celsius_to_F(val):
    return val * 9/5 + 32 

def mmday_to_inchyear(val, leap):
    if leap:
        return val * 0.0393701 * 366
    else:
        return val * 0.0393701 * 365

def get_observed():
    for item in zipped:
        
        slug = '_'.join(item)
        url = 'http://api.cal-adapt.org/api/series/' + slug + '/rasters/?pagesize=64&stat=mean&ref=/api/counties/34/'
        response = requests.get(url, headers=headers)

        # Get data
        if response.status_code == requests.codes.ok:
            print('Processing:', slug)
            data = response.json()
            # Create temp dataframe
            df = pd.DataFrame(data['results'])
            # Change format of `event` field to datetime
            df.event = pd.to_datetime(df.event, format='%Y-%m-%d')
            # The data from API should be sorted, but sort anyway
            df = df.sort_values('event')     
            # Convert units
            if 'tas' in slug:
                df.image = df.image.apply(lambda x: celsius_to_F(x))
            else:
                for i in range(len(df)):
                    year = df.iloc[i]['event'].year
                    if calendar.isleap(year):
                        df.iloc[i]['image'] = mmday_to_inchyear(df.iloc[i]['image'], True)
                    else:
                        df.iloc[i]['image'] = mmday_to_inchyear(df.iloc[i]['image'], False)

            series = df.image
            column_name = slug.replace("-", "").lower()
            observed_df[column_name] = series.values
        else:
            print('Failed:', slug)

In [206]:
get_observed()

Processing: tasmax_year_livneh
Processing: tasmin_year_livneh
Processing: pr_year_livneh


C:\Users\Shruti\Anaconda3\lib\site-packages\ipykernel\__main__.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Shruti\Anaconda3\lib\site-packages\ipykernel\__main__.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [184]:
observed_df.head()

,tasmax_year_livneh,tasmin_year_livneh,pr_year_livneh
1950-01-01,74.893606,47.813571,24.104610
1951-01-01,74.096737,46.416690,18.104470
1952-01-01,73.463145,46.995797,24.248145
1953-01-01,73.866604,46.923747,11.570093
1954-01-01,72.737019,46.808256,19.519974


In [96]:
def kelvin_to_F(val):
    return  (val - 273.15) * 9/5 + 32



def kgm2s_to_inchyear(val, leap):
    if (leap):
        return (val * 86400) * 0.0393701 * 366
    else:
        return (val * 86400) * 0.0393701 * 365



62.6

You can also create custom functions. The `image` values in our dataframe are in Kelvin and we want to change them to degrees Fahrenheit. Let's create a function that does this conversion and apply to each row in the `image` column using `.apply()`. An intro to [lambda](http://www.diveintopython.net/power_of_introspection/lambda_functions.html) functions in Python.

In [ ]:
df.image = df.image.apply(lambda K: (K - 273.15) * 9 / 5 + 32)
df.image.head()

### 4. Indexing

Each row in a DataFrame has a unique identifier called the `index`. By default Pandas autogenerates an integer index for each row. But it can be useful to idenitfy each row by other indices. 

In our DateFrame each row represents max. temp for a year. The year (`event` field) is a unique identifier. We can tell the DataFrame to use the `event` field as the  index field. This creates a timeseries and pandas provides a lot of [functionality](https://tomaugspurger.github.io/modern-7-timeseries.html) for working with timeseries data.

In [ ]:
# Change format of `event` field to datetime
df['event'] = pd.to_datetime(df['event'], format='%Y-%m-%d')
# Set event field as index
df = df.set_index(['event'])

In [ ]:
df.tail()

### 5. Filtering

In [ ]:
# Filter by value
hi_temps = df[df.image >= 80]
hi_temps.head()

# Filter by time. Uncomment the following lines and run the cell again
#filtered_df = df['20200101':'20300101']
#filtered_df

### 6. Resampling

Resampling is similar to a groupby: you split the time series into groups (10 year buckets below), apply a function to each group (mean), and combine the result (one row per group). This creates another data structure commonly used in Pandas called `Series` - a one-dimensional labeled array capable of holding any data type.

In [ ]:
# Only resample rows where year is between 2010 and 2099.
decadal_avg = df['20200101':'20990101'].image.resample("10AS").mean()
print(type(decadal_avg))
decadal_avg

In [ ]:
decadal_stats = df['20200101':'20990101'].image.resample("10AS").agg(['mean', 'max', 'min', 'std'])
print(type(decadal_stats))
decadal_stats